In [1]:
pip install transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
{"instruction": "...", "context": "...", "response": "..."}

{'instruction': '...', 'context': '...', 'response': '...'}

In [ ]:
from datasets import load_dataset
import json

# Load your Dolly dataset line by line to handle potential errors
data = []
with open("/content/databricks-dolly-15k.jsonl", "r") as f:
    for i, line in enumerate(f):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {i+1}: {e}")
            # Optionally, you can skip or handle the problematic line here

# If you successfully load the data, you can then create a dataset from the list of dictionaries
if data:
    from datasets import Dataset
    dataset = Dataset.from_list(data)

    # Check an example
    print(dataset[0])
else:
    print("No data was loaded from the file.")

{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.[3] It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.[4]", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
# Replace 'bert-base-uncased' with the model name you want to use
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    # Combine instruction, context, and response into a single text field
    texts = []
    for instruction, context, response in zip(examples["instruction"], examples["context"], examples["response"]):
        texts.append(instruction + " " + context + " " + response)
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False   # no masked LM, since GPT-style models are causal
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned-dolly",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    push_to_hub=False
)

In [ ]:
from transformers import AutoModelForCausalLM

# Load a pre-trained model for causal language modeling
# Replace 'gpt2' with the model name you want to use
model = AutoModelForCausalLM.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [ ]:
trainer.save_model("./finetuned-dolly")
tokenizer.save_pretrained("./finetuned-dolly")

('./finetuned-dolly/tokenizer_config.json',
 './finetuned-dolly/special_tokens_map.json',
 './finetuned-dolly/vocab.txt',
 './finetuned-dolly/added_tokens.json',
 './finetuned-dolly/tokenizer.json')

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./finetuned-dolly", tokenizer=tokenizer)
print(generator("Instruction: Write a short poem about the ocean\nResponse:", max_length=80))

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Instruction: Write a short poem about the ocean\nResponse: [unused74] [unused288] : [unused32] [unused467] nicaragua [unused282] [unused257] leans [unused10] [unused503] [unused313] [unused252] henry [unused281] [unused257] lanes plane choose [unused10] ʔ [unused321] [unused606] [unused479] [unused760] [unused279] [unused706] [unused282] [unused257] nevada [unused10] [unused479] [unused756] [unused279] south [unused506]vian [unused346] [unused506] [unused893] 〈 [unused12] games samuel ʔ [unused10] [unused361] watching samuel chance [unused497] [unused279] [unused418] [unused257] 〈 [unused10] [unused470] [unused334] [unused630] chance [unused497] [unused279] - [unused257] 〈 [unused12] [unused309] [unused831] [unused465] [unused760] [unused279] 版 [unused324] [unused611]vian [unused521] [unused378] lanes plane choose [unused313] [unused257] ᵉ [unused333] [unused744] makeup nevada [unused10] [unused285] : [unused32] [unused467] nicaragua ʔ [unused334] [unused313] 陳 [un

In [ ]:
!pip install transformers datasets accelerate evaluate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "distilgpt2"   # or any small causal LM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add a padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'text'],
        num_rows: 15015
    })
})


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/databricks-dolly-15k.jsonl")

def format_examples(example):
    if example["context"]:
        prompt = f"Instruction: {example['instruction']}\nContext: {example['context']}\nResponse: {example['response']}"
    else:
        prompt = f"Instruction: {example['instruction']}\nResponse: {example['response']}"
    return {"text": prompt}

dataset = dataset.map(format_examples)

# Tokenize the dataset after formatting
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [ ]:
trainer.save_model("/content/finetuned-dolly")
tokenizer.save_pretrained("/content/finetuned-dolly")

('/content/finetuned-dolly/tokenizer_config.json',
 '/content/finetuned-dolly/special_tokens_map.json',
 '/content/finetuned-dolly/vocab.json',
 '/content/finetuned-dolly/merges.txt',
 '/content/finetuned-dolly/added_tokens.json',
 '/content/finetuned-dolly/tokenizer.json')

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="/content/finetuned-dolly", tokenizer=tokenizer)

# Example instructions
print(generator("Instruction: Write a short poem about the ocean\nResponse:", max_length=80)[0]["generated_text"])
print(generator("Instruction: Classify this review: The movie was amazing and thrilling!\nResponse:", max_length=50)[0]["generated_text"])
print(generator("Instruction: Summarize: Artificial intelligence is transforming industries across the world.\nResponse:", max_length=60)[0]["generated_text"])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Instruction: Write a short poem about the ocean
Response:
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
Response:
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
Response:
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
Response:
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write a poem about the ocean." [sic]
"I'm going to write


Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Instruction: Classify this review: The movie was amazing and thrilling!
Response: Absolutely Awesome this movie was amazing!
Final Score: 8/10
- The Movie Was Awesome! - The Film Was Awesome! - The Film Was Awesome - The Film Was Awesome - The Film Was Awesome - The Film Was Awesome - The Film Was Awesome - The Film Was Awesome - The Film Was Awesome
Brief Summary: The movie is a story about a young man (who also happens to be a doctor) who falls in love with a group of women who are both extremely attractive and highly intelligent. He is convinced that they are all beautiful and when he decides on the name, they are all beautiful. The director doesn't want to make the movie too much for the audience, but it does have the potential to be entertaining. The film is very well made and really makes you want to try to enjoy it. While the movie is very well made and well done, it's not as well made and well done as The Star Wars movie. The plot will be very interesting and the characters wil

In [ ]:
# Example instruction for the generator
instruction = "Instruction: Write a short biography of Nikola Tesla\nResponse:"

# Use the generator pipeline to generate text based on the instruction
generated_text = generator(instruction, max_length=200)[0]["generated_text"]

# Print the generated text
print(generated_text)

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Instruction: Write a short biography of Nikola Tesla
Response: (I'm not sure what the first page of the book is, but I'm sure it's about a man who became an inventor. I'm sure it was written by a person who's been around long enough to know that he's a genius. )
I believe anyone can develop a long story, so I thought it would be interesting to dig up some of Nikola's writing. If you know of somebody, or you have an idea for a short story, I'd love to hear about it. If you think you know someone who might be interested, or if they have something to say, please let me know.
I'm quite interested in reading Nikola's own work, as well as my own personal experiences with his inventions and inventions, so if you want to know what makes Nikola such a great inventor, I'd love to hear about it.
I was a little surprised when I started reading that book, but I've been wanting to read it for some time. It's one of the best read articles I've read in a while, and it's really great, too.
I was actual

In [ ]:
from transformers import pipeline

# Load a pre-trained English to French translation model
translator = pipeline("translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr")

# Translate a sentence
english_sentence = "The stars shine at night."
french_translation = translator(english_sentence)[0]["translation_text"]

print(f"English: {english_sentence}")
print(f"French: {french_translation}")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


English: The stars shine at night.
French: Les étoiles brillent la nuit.
